In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_bu_city_mnf_submnf_dc_re_mode
  TBLPROPERTIES (
    'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
    'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
  )
  AS
  SELECT * EXCEPT (proc_date, upload_user, upload_time)
  FROM {catalog_name}.udp_wcm_silver_dataportal.d_cfg_bu_city_mnf_submnf_dc_re_mode
      WHERE upload_time = (SELECT MAX(upload_time) FROM {catalog_name}.udp_wcm_silver_dataportal.d_cfg_bu_city_mnf_submnf_dc_re_mode)
      QUALIFY ROW_NUMBER() OVER (PARTITION BY business_unit, city, manufacturer_id, sub_manufacturer_id, dc_id ORDER BY upload_time DESC) = 1 ;
""")